In [75]:
import requests
from datetime import datetime, timedelta
from pprint import pprint

In [76]:
# 측정소별 실시간 측정정보 조회 (매시 15분 내외 업데이트)
base_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
with open('keys/에어코리아api.txt') as file:
    service_key = file.read()       # 발급받은 에어코리아 API 키 입력

In [96]:
# 웹 요청시 같이 전달될 데이터 = 요청 메시지
params = {
        'serviceKey' : service_key,
        'returnType' : 'JSON',
        'numOfRows' : 30,
        'pageNo' : 1,
        'stationName' : "서대문구", # 시도 이름
        'dataTerm' : 'DAILY',
        'ver' : "1.4" # 오퍼레이션 버전
    }

res = requests.get(url=base_url , params=params)
# print(res.status_code, type(res.text), res.url)
# print()
# print(res.text)

# 응답 데이터 정리
from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
data = res.json() # json.loads(res.text)와 같은 기능
data = data['response']['body']['items']
pprint(data)

[{'coFlag': None,
  'coGrade': '1',
  'coValue': '1.2',
  'dataTime': '2023-11-23 14:00',
  'khaiGrade': '4',
  'khaiValue': '285',
  'mangName': '도시대기',
  'no2Flag': None,
  'no2Grade': '1',
  'no2Value': '0.021',
  'o3Flag': None,
  'o3Grade': '2',
  'o3Value': '0.047',
  'pm10Flag': None,
  'pm10Grade': '3',
  'pm10Grade1h': '3',
  'pm10Value': '143',
  'pm10Value24': '113',
  'pm25Flag': None,
  'pm25Grade': '3',
  'pm25Grade1h': '3',
  'pm25Value': '74',
  'pm25Value24': '71',
  'so2Flag': None,
  'so2Grade': '1',
  'so2Value': '0.004',
  'stationCode': '111191',
  'stationName': '서대문구'},
 {'coFlag': None,
  'coGrade': '1',
  'coValue': '1.2',
  'dataTime': '2023-11-23 13:00',
  'khaiGrade': '4',
  'khaiValue': '289',
  'mangName': '도시대기',
  'no2Flag': None,
  'no2Grade': '1',
  'no2Value': '0.022',
  'o3Flag': None,
  'o3Grade': '2',
  'o3Value': '0.046',
  'pm10Flag': None,
  'pm10Grade': '3',
  'pm10Grade1h': '3',
  'pm10Value': '145',
  'pm10Value24': '109',
  'pm25Flag': None

In [95]:
# 완성코드

def get_air_quality(station_Name):
    
    # 웹 요청시 같이 전달될 데이터 = 요청 메시지
    params = {
        'serviceKey' : service_key,
        'returnType' : 'JSON',
        'numOfRows' : 30,
        'pageNo' : 1,
        'stationName' : station_Name, # 시도 이름
        'dataTerm' : 'DAILY',
        'ver' : "1.4" # 오퍼레이션 버전
    }

    res = requests.get(url=base_url , params=params)

    # 응답 데이터 정리
    from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
    data = res.json() # json.loads(res.text)와 같은 기능
    data = data['response']['body']['items']
    
    # 데이터를 날짜와 시간에 대한 기준으로 정렬
    sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

    # 최신 데이터 선택
    latest_data = sorted_data[0]
    
    # 최종 데이터 생성
    results = {
        '날짜': latest_data['dataTime'],
        '이름': latest_data['stationName'],
        '측정망 정보': latest_data['mangName'],
        '아황산가스 농도': latest_data['so2Value'] + "ppm",
        '일산화탄소 농도': latest_data['coValue'] + "ppm",
        '오존 농도': latest_data['o3Value'] + "ppm",
        '이산화질소 농도': latest_data['no2Value'] + "ppm",
        '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
        '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
    }

    # pprint(results)
    return results
    
# 테스트 코드
if __name__ == "__main__":
    print(get_air_quality('서대문구'))

{'날짜': '2023-11-23 14:00', '이름': '서대문구', '측정망 정보': '도시대기', '아황산가스 농도': '0.004ppm', '일산화탄소 농도': '1.2ppm', '오존 농도': '0.047ppm', '이산화질소 농도': '0.021ppm', '미세먼지(PM10) 농도': '143ug/m³', '초미세먼지(PM2.5) 농도': '74ug/m³'}
